# University of Canberra
#### 11482 - Pattern Recognition and Machine Learning
#### Tutorial Class Thursday 0930 (Semester 2 / 2024)
#### Assignment Stage 2
#### Group 101 (Individual)
#### James McGuinness


### B1

- Select a dataset for your project
- You may use the 10 times rule - if I have 10 parameters, then I should have 10 x 10 = 100 images

In [2]:
# Pre-reqs and libraries to import
import sys
sys.path.append('C:\\Users\\James\\Desktop\\RoboticsJourney\\Self_Study\\1.PatternReg.ML\\introduction_to_ml_with_python')
import mglearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [4]:
dir(cancer)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [44]:
print(f"{len(cancer.data)} samples")
print(f"{len(cancer.feature_names)} features per sample")
print(f"The features are:\n{cancer.feature_names}")
print(f"I think this is the first sample:\n{cancer.data[1]}")
print(f"The target is either one or zero {cancer.target[0:4]}")
print(f"The target names are : {cancer.target_names}")
print(f"I think class 1 would mean positive and would be 'malignant'")
print("Sample counts per class:\n{}".format(
    {n: v for n, v in zip(cancer.target_names, np.bincount(cancer.target))}))

569 samples
30 features per sample
The features are:
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
I think this is the first sample:
[2.057e+01 1.777e+01 1.329e+02 1.326e+03 8.474e-02 7.864e-02 8.690e-02
 7.017e-02 1.812e-01 5.667e-02 5.435e-01 7.339e-01 3.398e+00 7.408e+01
 5.225e-03 1.308e-02 1.860e-02 1.340e-02 1.389e-02 3.532e-03 2.499e+01
 2.341e+01 1.588e+02 1.956e+03 1.238e-01 1.866e-01 2.416e-01 1.860e-01
 2.750e-01 8.902e-02]
The target is either one or zero [0 0 0 0]
The targe

### B2

- Analyze data
- Visualization to understand the data

### B3

- Prepare data
- Use data transforms / scaling / re-shaping
- Feature selection in order to reduce dimensionality 
- Summarize data 

In [49]:
# Could do regularization here - ref page 61 with graph ic features and C parameter - L1 regularization penalty
# MinMaxScaler - page 134
# PCA on page 140 - with visualisation of all properties in a histogram on page 143
# PCA visual on page 145 - showing malignant and benign


### B4

- Decide which algorithms to use
- Explain my reasoning
- Select four algorithms to spot-check 
- Determine the best based on accuracy evaluations

In [47]:
# Decision tree ref page 75 
# I think SVM on page 100?
# Can I do some Polynomial Linear Regression here?
# Neural Networks on page 116 with heatmap

### B5

- Write and submit report of 2 pages